In [1]:
from sympy import *
init_printing()

In [32]:
def printcode(vs, es, names):
    for var, val in vs:
        print '    %s = %s' % (var, val)
    for i, val in enumerate(es):
        print '    %s = %s' % (names[i], val)

In [49]:
# Landmark state (just its position)
lx, ly = symbols("l_x l_y")
X = Matrix([lx, ly])

# Particle state variables
x, y, theta = symbols("p_x p_y theta", real=True)

# Bearing measurement of landmark
def bearing_measurement():
    s, c = sin(theta), cos(theta)
    # Note: the Y coordinate of the rotation matrix is flipped here
    # because the relative bearing of the cone w.r.t. the center of the camera is flipped
    # or something, I forget
    R = Matrix([[c, s], [s, -c]])
    lo = R * Matrix([lx - x, ly - y])
    return Matrix([atan2(lo[1], lo[0])])

h_x_bearing = bearing_measurement()
l_px = symbols("l_px")
h_z_bearing = Matrix([l_px])
z_bearing = Matrix([l_px])
R_bearing = symbols("lm_R")


def generate_measurement_1d(X, h_x, h_z, z_k):
    H = h_x.jacobian(X)
    M = h_z.jacobian(z_k) + h_x.jacobian(z_k)
    y_k = h_z - h_x
    #vs, es = cse([y_k, H, M], optimizations='basic',
    #                symbols=numbered_symbols("k"))
    #return vs, es
    p11, p12, p22 = symbols("p11 p12 p22")
    r = symbols("r")
    P = Matrix([[p11, p12], [p12, p22]])
    S = H*P*H.T + Matrix([[r]])
    LL = -y_k[0,0]**2/S[0,0] - 0.5*log((2*pi)**2 * S[0,0])
    K = P*H.T / S[0,0]
    Pnew = (eye(2) - K*H)*P
    Pvec = Matrix([Pnew[0,0], Pnew[0,1], Pnew[1,1]])
    #vs, es = cse([y_k, K, LL, Pvec], optimizations='basic',
    #                symbols=numbered_symbols("k"))
    vs, es = cse([y_k[0], S[0,0], H[0,0], H[0,1], LL], # optimizations='basic',
                    symbols=numbered_symbols("k"))
    return vs, es, ["y_k", "S", "H1", "H2", "LL"]

printcode(*generate_measurement_1d(X, h_x_bearing, h_z_bearing, z_bearing))

    k0 = sin(theta)
    k1 = l_x - p_x
    k2 = k0*k1
    k3 = cos(theta)
    k4 = l_y - p_y
    k5 = k3*k4
    k6 = k2 - k5
    k7 = k0*k4 + k1*k3
    k8 = l_px - atan2(k6, k7)
    k9 = 1/(k6**2 + k7**2)
    k10 = k7*k9
    k11 = k9*(-k2 + k5)
    k12 = k0*k10 + k11*k3
    k13 = k0*k11 - k10*k3
    k14 = k12*(k12*p11 + k13*p12) + k13*(k12*p12 + k13*p22) + r
    y_k = k8
    S = k14
    H1 = k12
    H2 = k13
    LL = -0.5*log(4*pi**2*k14) - k8**2/k14


In [40]:
def generate_xP():
    p11, p12, p22, H1, H2, S, yk = symbols("p11 p12 p22 H1 H2 S y_k")
    P = Matrix([[p11, p12], [p12, p22]])
    H = Matrix([[H1, H2]])
    K = P*H.T / S
    dx = K*yk
    Pnew = (eye(2) - K*H)*P
    
    vs, es = cse([dx[0], dx[1], Pnew[0, 0], Pnew[0,1], Pnew[1,1]], optimizations='basic',
                    symbols=numbered_symbols("k"))
    return vs, es, ["dx", "dy", "p11", "p12", "p22"]

printcode(*generate_xP())

    k0 = 1/S
    k1 = H2*p12
    k2 = k0*(H1*p11 + k1)
    k3 = H1*p12
    k4 = H2*p22
    k5 = k0*(k3 + k4)
    k6 = H1*k2 - 1
    dx = k2*y_k
    dy = k5*y_k
    p11 = -k1*k2 - k6*p11
    p12 = -k2*k4 - k6*p12
    p22 = -k3*k5 - p22*(H2*k5 - 1)
